In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

In [6]:
aapl = yf.download("aapl", interval="1d", period="2y",auto_adjust=True)

[*********************100%***********************]  1 of 1 completed


In [11]:
aapl

Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2023-02-08,150.339050,152.971373,149.596855,152.278660,64120100
2023-02-09,149.299957,152.723957,148.854643,152.179678,56007100
2023-02-10,149.666687,149.993753,147.892617,148.130487,57450700
2023-02-13,152.481430,152.887771,149.577486,149.607218,62199000
2023-02-14,151.837204,152.402141,149.518023,150.766809,61707600
...,...,...,...,...,...
2025-02-03,228.009995,231.830002,225.699997,229.990005,73063300
2025-02-04,232.800003,233.130005,226.649994,227.250000,45067300
